<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#Cleanup-ATC-file" data-toc-modified-id="Cleanup-ATC-file-4.1">Cleanup ATC file</a></span></li><li><span><a href="#Clean-up-NCR-file" data-toc-modified-id="Clean-up-NCR-file-4.2">Clean up NCR file</a></span></li><li><span><a href="#Clean-up-T-cells-file" data-toc-modified-id="Clean-up-T-cells-file-4.3">Clean up T cells file</a></span></li><li><span><a href="#Create-master-table-for-clinical-and-flow-data" data-toc-modified-id="Create-master-table-for-clinical-and-flow-data-4.4">Create master table for clinical and flow data</a></span></li><li><span><a href="#Cohort-cleanup" data-toc-modified-id="Cohort-cleanup-4.5">Cohort cleanup</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(ggplot2)
library(gridExtra)
library(ComplexHeatmap)
library(circlize)

Loading required package: grid

ComplexHeatmap version 2.6.2
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


Warning message:
“package ‘circlize’ was built under R version 4.0.5”
circlize version 0.4.14
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circli

# Functions

# Paths

In [2]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

All csv files are from "Toronto expansion and NCR data 4.xlsx"

## Cleanup ATC file

On excel, copy row 2:end of "ATC count" sheet to a new spreadsheet, adjust columns:
" ID#	Age		FOLD	ATC	FOLD1	ATC1	FOLD2	ATC2	FOLD3	ATC3	FOLD4	ATC4"
save as ATC_count.csv

In [3]:
ATCs <- read.csv(paste0(datapath, "Flow/ATC_count.csv"),
                 header = T, check.names = F, stringsAsFactors = F, 
                 na.strings = c("NA","", "#DIV/0!","#VALUE!","#REF!"))

In [4]:
head(ATCs[221:338,])

,ID#,Age,,FOLD,ATC,FOLD1,ATC1,FOLD2,ATC2,FOLD3,ATC3,FOLD4,ATC4
,<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
221,320,NA,AML,8.9,0.00,3.5,0.00,13.2,0.00,x,0.00,0,0.0
222,321,NA,HD,0,0.00,0,0.00,0,0.00,0,0.00,0,0.0
223,322,NA,DLBCL,1.1,0.00,0.4,0.00,4.3,0.00,4.7,0.00,1,0.0
224,323,NA,HR ALL,0,154.75,0,1397.28,0,786.24,0,563.68,0,979.5
225,324,NA,T-ALL,0,0.00,0,0.00,0,0.00,0,0.00,0,0.0
226,325,NA,ERMS,0,NA,0,927.46,0,624.72,0,163.77,0,0.0


Subset to include patients 330 to 437

In [5]:
ATC_subset <- ATCs[221:338,]

In [6]:
ATCssubset1 <- ATC_subset[,c("FOLD", "ATC")]
ATCssubset2 <- ATC_subset[,c("FOLD1", "ATC1")]
ATCssubset3 <- ATC_subset[,c("FOLD2", "ATC2")]
ATCssubset4 <- ATC_subset[,c("FOLD3", "ATC3")]
ATCssubset5 <- ATC_subset[,c("FOLD4", "ATC4")]

In [7]:
ATCssubset1 <- cbind(ATC_subset$` ID#`, ATCssubset1)
ATCssubset2 <- cbind(ATC_subset$` ID#`, ATCssubset2)
ATCssubset3 <- cbind(ATC_subset$` ID#`, ATCssubset3)
ATCssubset4 <- cbind(ATC_subset$` ID#`, ATCssubset4)
ATCssubset5 <- cbind(ATC_subset$` ID#`, ATCssubset5)

In [8]:
colnames(ATCssubset1) <- c("Patient", "FOLD", "ATC")
colnames(ATCssubset2) <- c("Patient", "FOLD", "ATC")
colnames(ATCssubset3) <- c("Patient", "FOLD", "ATC")
colnames(ATCssubset4) <- c("Patient", "FOLD", "ATC")
colnames(ATCssubset5) <- c("Patient", "FOLD", "ATC")

In [9]:
ATCssubset1$cycle <- "01"
ATCssubset2$cycle <- "02"
ATCssubset3$cycle <- "03"
ATCssubset4$cycle <- "04"
ATCssubset5$cycle <- "05"

In [10]:
myATCdf <- rbind(ATCssubset1, ATCssubset2,ATCssubset3, ATCssubset4,ATCssubset5)

In [11]:
head(myATCdf)

,Patient,FOLD,ATC,cycle
,<int>,<chr>,<dbl>,<chr>
221,320,8.9,0.00,01
222,321,0,0.00,01
223,322,1.1,0.00,01
224,323,0,154.75,01
225,324,0,0.00,01
226,325,0,NA,01


In [12]:
write.csv(myATCdf, file = paste(datapath, "ATC_df.csv", sep = ""),row.names = F)

## Clean up NCR file

Copy "NCRs pre" sheet to a new spreadsheet (values only), save as NCRs.csv

In [13]:
NCRs <- read.csv(paste0(datapath, "Flow/NCRs.csv"),header = T, check.names = F, stringsAsFactors = F, 
                 na.strings = c("NA","", "#DIV/0!","#VALUE!","#REF!"))

In [14]:
head(NCRs[221:nrow(NCRs),])

,ID#,Age,DX,TE Fold,CD3,PD-1,PDL1,LAG3,TIM3,CTLA4,⋯,LAG3,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3
,<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<int>,⋯,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
221,320,6,AML,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
222,321,16,HD,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
223,322,18,DLBCL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
224,323,1,HR ALL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
225,324,12,T-ALL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
226,325,2,ERMS,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Subset to include patients 330 to 437

In [15]:
NCRssubset <- NCRs[221:nrow(NCRs),]

In [16]:
head(NCRssubset)

,ID#,Age,DX,TE Fold,CD3,PD-1,PDL1,LAG3,TIM3,CTLA4,⋯,LAG3,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3
,<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<int>,⋯,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
221,320,6,AML,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
222,321,16,HD,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
223,322,18,DLBCL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
224,323,1,HR ALL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
225,324,12,T-ALL,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
226,325,2,ERMS,NA,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Get empty columns. They indicate serial samplesets for each patient

In [17]:
subsets <- grep("^$", colnames(NCRssubset))

In [18]:
subsets

[1]  18  34  50  66  82  98 114 130 146 162 178 194 210 226 242 258 274 290

Save each sampleset then rbind

In [19]:
NCRssubset1 <- NCRssubset[,1:17]
NCRssubset2 <- NCRssubset[,c(1:3,19:33)]
NCRssubset3 <- NCRssubset[,c(1:3,35:49)]
NCRssubset4 <- NCRssubset[,c(1:3,51:65)]
NCRssubset5 <- NCRssubset[,c(1:3,67:81)]

In [20]:
NCRssubset1$`Pass?` <- NA

In [21]:
NCRssubset1$cycle <- "01"
NCRssubset2$cycle <- "02"
NCRssubset3$cycle <- "03"
NCRssubset4$cycle <- "04"
NCRssubset5$cycle <- "05"

In [22]:
myNCRdf <- rbind(NCRssubset1,
                NCRssubset2,
                NCRssubset3,
                NCRssubset4,
                NCRssubset5)

Reorder columns and colnames cleanup

In [23]:
myNCRdf <- myNCRdf[, c(1,19,2:4,18,5:17)]

In [24]:
colnames(myNCRdf)[c(1,4,5,6)] <- c("Patient", "Disease_type", "TE_fold", "Pass")

Assign -1 to all "x" values. These are samples that failed the flow experiment

In [25]:
myNCRdf$`PD-1` <- gsub("x", -1, myNCRdf$`PD-1`)
myNCRdf$LAG3 <- gsub("x", -1, myNCRdf$LAG3)
myNCRdf$TIM3 <- gsub("x", -1, myNCRdf$TIM3)

In [26]:
myNCRdf$`PD-1` <- as.numeric(myNCRdf$`PD-1`)
myNCRdf$LAG3 <- as.numeric(myNCRdf$LAG3)
myNCRdf$TIM3 <- as.numeric(myNCRdf$TIM3)

In [27]:
head(myNCRdf)

,Patient,cycle,Age,Disease_type,TE_fold,Pass,CD3,PD-1,PDL1,LAG3,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3
,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
221,320,01,6,AML,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
222,321,01,16,HD,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
223,322,01,18,DLBCL,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
224,323,01,1,HR ALL,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
225,324,01,12,T-ALL,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
226,325,01,2,ERMS,NA,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [28]:
dim(myNCRdf)

[1] 590  19

Generate percentage NCRs

In [29]:
myNCRdf$`PD1%` <- (myNCRdf$`PD-1`/myNCRdf$CD3)* 100
myNCRdf$`PDL1%` <- (myNCRdf$PDL1/myNCRdf$CD3)* 100
myNCRdf$`LAG3%` <- (myNCRdf$LAG3/myNCRdf$CD3)* 100
myNCRdf$`TIM3%` <- (myNCRdf$TIM3/myNCRdf$CD3)* 100
myNCRdf$`CTLA4%` <- (myNCRdf$CTLA4/myNCRdf$CD3)* 100

In [30]:
write.csv(myNCRdf,file = paste0(datapath, "NCR_df.csv"),row.names = F)

## Clean up T cells file


Copy "Pre Memory Subsets" sheet to a new spreadsheet (values only)

move the first 3 colnames up to align the the other colnames
remove row 2. save as Tsubsets.csv

In [31]:
Tcells <- read.csv(paste0(datapath, "Flow/Tsubsets.csv"),header = T, 
                 check.names = F, stringsAsFactors = F,
                 na.strings = c("NA","","#DIV/0!","#VALUE!","#REF!"))

In [32]:
head(Tcells)

,ID#,Age,DX,TE Fold,Pass?,CD3,EM,CM,Naïve,SCM,⋯,Naïve,SCM,TE,EM%,CM%,Naïve%,SCM%,TE%,Naïve/TE,N+CM+SCM/TE+EM
,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,100,22,BMT,0.53,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,101,6,SR ALL,26.4,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,102,6,SR ALL,8.11,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,103,4,Burkitt's Lymphoma,3.2,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,104,1,HR ALL,22.86,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,105,14,HR ALL,30.91,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Subset to include patients 330 to 437

In [33]:
Tcellsubset <- Tcells[221:nrow(Tcells),]

Get empty columns and seperate serial samples

In [34]:
grep("^$", colnames(Tcellsubset))

[1]  19  35  51  67  83  99 115 131 147 163 179 195 211 227 243 259 275 291

In [35]:
Tcellsubset1 <- Tcellsubset[,1:18]
Tcellsubset2 <- Tcellsubset[,c(1:3,20:34)]
Tcellsubset3 <- Tcellsubset[,c(1:3,36:50)]
Tcellsubset4 <- Tcellsubset[,c(1:3,52:66)]
Tcellsubset5 <- Tcellsubset[,c(1:3,68:82)]

In [36]:
Tcellsubset1$cycle <- "01"
Tcellsubset2$cycle <- "02"
Tcellsubset3$cycle <- "03"
Tcellsubset4$cycle <- "04"
Tcellsubset5$cycle <- "05"

In [37]:
myTcellsdf <- rbind(Tcellsubset1,
                Tcellsubset2,
                Tcellsubset3,
                Tcellsubset4,
                Tcellsubset5)

In [38]:
myTcellsdf <- myTcellsdf[, c(1,19,2:4,5:18)]

In [39]:
colnames(myTcellsdf)[c(1,4,5,6)] <- c("Patient", "Disease_type", "TE_fold", "Pass")

In [40]:
head(myTcellsdf)

,Patient,cycle,Age,Disease_type,TE_fold,Pass,CD3,EM,CM,Naïve,SCM,TE,EM%,CM%,Naïve%,SCM%,TE%,Naïve/TE,N+CM+SCM/TE+EM
,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
221,320,01,6,AML,8.9,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
222,321,01,16,HD,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
223,322,01,18,DLBCL,1.1,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
224,323,01,1,HR ALL,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
225,324,01,12,T-ALL,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
226,325,01,2,ERMS,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [41]:
write.csv(myTcellsdf,file = paste0(datapath, "Tcells_df.csv"),
         row.names = F)

## Create master table for clinical and flow data

In [42]:
Tcells <- read.csv(paste0(datapath, "Tcells_df.csv"),
                 header = T, stringsAsFactors = F,check.names = F)

In [43]:
NCRs <- read.csv(paste0(datapath, "NCR_df.csv"),
                 header = T, stringsAsFactors = F,check.names = F)

In [44]:
ATCs <- read.csv(paste0(datapath, "ATC_df.csv"),
                 header = T, stringsAsFactors = F,check.names = F)

In [45]:
ATC_Tcells <- merge(ATCs, Tcells, by = c("Patient", "cycle"))

In [46]:
dim(ATCs)
dim(Tcells)
dim(ATC_Tcells)

[1] 590   4

[1] 590  19

[1] 590  21

In [47]:
ATC_Tcells_NCR <- merge(ATC_Tcells, NCRs[,!colnames(NCRs)%in% c("Disease_type", "Age", "TE_fold", "Pass", "CD3")], by = c("Patient", "cycle"))

In [48]:
head(ATC_Tcells_NCR[,5:15])

,Age,Disease_type,TE_fold,Pass,CD3,EM,CM,Naïve,SCM,TE,EM%
,<dbl>,<chr>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,6,AML,8.9,NA,0,NA,NA,NA,NA,NA,NA
2,6,AML,3.5,NA,0,NA,NA,NA,NA,NA,NA
3,6,AML,13.2,NA,0,NA,NA,NA,NA,NA,NA
4,6,AML,x,NA,0,NA,NA,NA,NA,NA,NA
5,6,AML,0,NA,0,NA,NA,NA,NA,NA,NA
6,16,HD,0,NA,0,NA,NA,NA,NA,NA,NA


In [49]:
ATC_Tcells_NCR$FOLD[ATC_Tcells_NCR$FOLD == "x"] <- NA

In [50]:
cols <- c("FOLD", "EM", "CM", "SCM", "TE", "Naïve")
ATC_Tcells_NCR[,cols] <- apply(ATC_Tcells_NCR[,cols],2, function(x) as.numeric(as.character(x)))

Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”


In [51]:
colnames(ATC_Tcells_NCR)

[1] "Patient"           "cycle"             "FOLD"             
 [4] "ATC"               "Age"               "Disease_type"     
 [7] "TE_fold"           "Pass"              "CD3"              
[10] "EM"                "CM"                "Naïve"            
[13] "SCM"               "TE"                "EM%"              
[16] "CM%"               "Naïve%"            "SCM%"             
[19] "TE%"               "Naïve/TE"          "N+CM+SCM/TE+EM"   
[22] "PD-1"              "PDL1"              "LAG3"             
[25] "TIM3"              "CTLA4"             "PD1%"             
[28] "PDL1%"             "LAG3%"             "TIM3%"            
[31] "CTLA4%"            "Any NCR"           "PD1 + LAG3 + TIM3"

In [52]:
head(ATC_Tcells_NCR)

,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,Pass,CD3,EM,⋯,LAG3,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3
,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,320,1,8.9,0,6,AML,8.9,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,320,2,3.5,0,6,AML,3.5,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,320,3,13.2,0,6,AML,13.2,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,320,4,NA,0,6,AML,x,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,320,5,0.0,0,6,AML,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,321,1,0.0,0,16,HD,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [53]:
ATC_Tcells_NCR$cycle[ATC_Tcells_NCR$cycle == 1] <- "01"
ATC_Tcells_NCR$cycle[ATC_Tcells_NCR$cycle == 2] <- "02"
ATC_Tcells_NCR$cycle[ATC_Tcells_NCR$cycle == 3] <- "03"
ATC_Tcells_NCR$cycle[ATC_Tcells_NCR$cycle == 4] <- "04"
ATC_Tcells_NCR$cycle[ATC_Tcells_NCR$cycle == 5] <- "05"

In [54]:
head(ATC_Tcells_NCR)

,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,Pass,CD3,EM,⋯,LAG3,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,320,01,8.9,0,6,AML,8.9,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,320,02,3.5,0,6,AML,3.5,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,320,03,13.2,0,6,AML,13.2,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,320,04,NA,0,6,AML,x,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,320,05,0.0,0,6,AML,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,321,01,0.0,0,16,HD,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Cohort cleanup

In [55]:
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Hepatoblastoma"] <- "HB"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Ewings"] <- "EWS"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Burkitts"] <- "BL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Burkitts "] <- "BL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Burkitt's"] <- "BL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "Wilms"] <- "WILMS"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "B cell lymphoma"] <- "BLL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "B lymphoblastic lymphoma"] <- "BLL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "NBL"] <- "NB"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "aLL"] <- "ALL"

Typos according to DB

In [56]:
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "DLMCL"] <- "DLBCL"
ATC_Tcells_NCR$Disease_type[ATC_Tcells_NCR$Disease_type == "SML"] <- "CML"

In [57]:
table(ATC_Tcells_NCR$Disease_type)


  ALCL    ALL    AML   ARMS     BL    BLL    CML  DLBCL   ERMS    EWS     HB 
    10    115     55      5     20     10     20     15     25     30     20 
    HD HR ALL  MS NB     NB     OS  PMBCL    RMS SR ALL  T-ALL  WILMS 
    40     65      5     20     60     10     15     15     20     15 

In [58]:
leuk <- c("T-ALL", "SR ALL", "HR ALL", "ALL", "AML", "CML")
lymph <- c("ALCL", "BL", "BLL", "PMBCL", "HD", "CLBCL", "DLBCL")
solid <- c("ARMS", "EWS", "HB", "MS NB", "NB", "OS", "RMS", "WILMS", "ERMS")

In [59]:
ATC_Tcells_NCR$cancergroup[ATC_Tcells_NCR$Disease_type %in% leuk] <- "Leukemia"
ATC_Tcells_NCR$cancergroup[ATC_Tcells_NCR$Disease_type %in% lymph] <- "Lymphoma"
ATC_Tcells_NCR$cancergroup[ATC_Tcells_NCR$Disease_type %in% solid] <- "Solid"

In [61]:
table(ATC_Tcells_NCR$cancergroup, useNA = "always")


Leukemia Lymphoma    Solid     <NA> 
     290      105      195        0 

In [62]:
head(ATC_Tcells_NCR)

,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,Pass,CD3,EM,⋯,TIM3,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3,cancergroup
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,320,01,8.9,0,6,AML,8.9,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia
2,320,02,3.5,0,6,AML,3.5,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia
3,320,03,13.2,0,6,AML,13.2,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia
4,320,04,NA,0,6,AML,x,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia
5,320,05,0.0,0,6,AML,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia
6,321,01,0.0,0,16,HD,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Lymphoma


In [63]:
ATC_Tcells_NCR$Patient <- paste("CHP_", ATC_Tcells_NCR$Patient, sep = "")

In [64]:
ATC_Tcells_NCR$sample_id <- paste( ATC_Tcells_NCR$Patient, "_", ATC_Tcells_NCR$cycle, sep = "")

In [65]:
head(ATC_Tcells_NCR)

,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,Pass,CD3,EM,⋯,CTLA4,PD1%,PDL1%,LAG3%,TIM3%,CTLA4%,Any NCR,PD1 + LAG3 + TIM3,cancergroup,sample_id
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,CHP_320,01,8.9,0,6,AML,8.9,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia,CHP_320_01
2,CHP_320,02,3.5,0,6,AML,3.5,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia,CHP_320_02
3,CHP_320,03,13.2,0,6,AML,13.2,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia,CHP_320_03
4,CHP_320,04,NA,0,6,AML,x,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia,CHP_320_04
5,CHP_320,05,0.0,0,6,AML,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Leukemia,CHP_320_05
6,CHP_321,01,0.0,0,16,HD,0,NA,0,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,Lymphoma,CHP_321_01


In [66]:
write.csv(ATC_Tcells_NCR,
         file = paste0(datapath, "INT_metadata_flow.csv"),row.names = F)